In [2]:
from PIL import Image,ImageFilter, ImageEnhance
import torch
import torch.nn as nn
import torch.optim as optim

In [66]:
# Определение модели Сиамской сети
class SiameseNetwork(nn.Module):
    def __init__(self):
        super(SiameseNetwork, self).__init__()
        
        # Спецификация сверточных слоев
        self.convnet = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=5),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(32),
            nn.MaxPool2d(2, stride=2),

            nn.Conv2d(32, 64, kernel_size=5),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(64),
            nn.MaxPool2d(2, stride=2)
        )
        
        # Полносвязные слои для классификации
        self.fc = nn.Sequential(
            nn.Linear(64*53*53, 256),  # Примерный размер должен быть предварительно вычислен
            nn.ReLU(inplace=True),
            nn.Linear(256, 256),
            nn.ReLU(inplace=True),
            nn.Linear(256, 2)
        )

    def forward_once(self, x):
        output = self.convnet(x)
        output = output.view(output.size()[0], -1)
        output = self.fc(output)
        return output

    def forward(self, input1, input2):
        output1 = self.forward_once(input1)
        output2 = self.forward_once(input2)
        return output1, output2

# Функция потерь - Contrastive Loss
class ContrastiveLoss(nn.Module):
    def __init__(self, margin=2.0):
        super(ContrastiveLoss, self).__init__()
        self.margin = margin

    def forward(self, output1, output2, label):
        euclidean_distance = nn.functional.pairwise_distance(output1, output2)
        loss_contrastive = torch.mean((1-label) * torch.pow(euclidean_distance, 2) +
                                      (label) * torch.pow(torch.clamp(self.margin - euclidean_distance, min=0.0), 2))
        return loss_contrastive

# Инициализация модели и определение оптимизатора и потерь
model = SiameseNetwork()
criterion = ContrastiveLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Этот фрагмент кода представляет собой базовую сеть и требует процедур обучения, загрузки данных и тестирования.

In [11]:
def image_input_convert(path):

    image = Image.open(path)

    # Конвертация в градации серого
    gray_image = image.convert('L')

    # Повышение контрастности
    enhancer = ImageEnhance.Contrast(gray_image)
    contrast_image = enhancer.enhance(2.0)  # Уровень контрастности может быть настроен

    # Применение фильтра увеличения резкости
    sharp_image = contrast_image.filter(ImageFilter.SHARPEN)

    # Применение фильтра высоких частот (опционально)
    high_pass_image = sharp_image.filter(ImageFilter.FIND_EDGES)

    return high_pass_image


In [12]:
im = image_input_convert('Data/1.jpeg')
im.show()

In [62]:
im_pattern  = image_input_convert('traf.jpg')
im_pattern.show()

In [11]:
from torch.utils.data import Dataset, DataLoader
import numpy as np

class SiameseNetworkDataset(Dataset):
    def __init__(self, image_folder, transform=None):
        # Здесь должен быть ваш код для загрузки и обработки данных
        # В этом примере просто используются случайные данные
        self.image_folder = image_folder
        self.transform = transform
        self.data = [torch.rand(1, 105, 105) for _ in range(1000)]
        self.labels = np.random.randint(0, 2, 1000)

    def __getitem__(self, index):
        img1 = self.data[index]
        target = self.labels[index]
        # Найти второе изображение
        if target == 1:
            # Найти изображение того же класса
            index2 = (index + np.random.randint(1, 100)) % 1000
        else:
            # Найти изображение другого класса
            index2 = (index + np.random.randint(1, 100)) % 1000

        img2 = self.data[index2]

        if self.transform:
            img1 = self.transform(img1)
            img2 = self.transform(img2)
        
        return img1, img2, torch.from_numpy(np.array([target], dtype=np.float32))

    def __len__(self):
        return len(self.data)

# Создать DataLoader
dataset = SiameseNetworkDataset('./data')
dataloader = DataLoader(dataset, shuffle=True, num_batch=8)

def train_epoch(model, dataloader, criterion, optimizer, device='cpu'):
    model.train()
    running_loss = 0.0
    
    for data in dataloader:
        img1, img2, labels = data
        img1, img2, labels = img1.to(device), img2.to(device), labels.to(device)
        
        optimizer.zero_grad()
        output1, output2 = model(img1, img2)
        loss = criterion(output1, output2, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()

    average_loss = running_loss / len(dataloader)
    return average_loss

# Обучение модели
num_epochs = 10
for epoch in range(num_epochs):
    loss = train_epoch(model, dataloader, criterion, optimizer)
    print(f'Epoch {epoch+1}, Loss: {loss}')




In [7]:
final = image.filter(ImageFilter.Kernel((3, 3), (-1, -1, -1, -1, 8,
                                          -1, -1, -1, -1), 1, 0))

final.show()